# Land surface temprature (LST)  data 

In [7]:
import ee
import pandas as pd
import os

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project="gee-ndvi-berhe")

# Define location
latitude, longitude = 8.4281, 33.7833  # Lankien, South Sudan
roi = ee.Geometry.Point([longitude, latitude])

# Define start and end date
start_date = '2015-01-01'
end_date = '2024-12-31'

# Load MODIS Land Surface Temperature (LST) dataset
modis_lst = ee.ImageCollection('MODIS/061/MOD11A1') \
    .select(['LST_Day_1km', 'QC_Day']) \
    .filterDate(start_date, end_date) \
    .filterBounds(roi)

# Function to extract data
def extract_data(image):
    date = image.date().format("YYYY-MM-dd")  # Get date as string
    
    # Reduce region to extract LST value at the point
    reduced = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=1000,
        bestEffort=True
    )
    
    # Extract the LST value, checking if it exists
    lst_value = reduced.get('LST_Day_1km')
    
    # Return a feature with the date and extracted value
    return ee.Feature(None, {'date': date, 'LST_Day_1km': lst_value})

# Apply extraction
features = modis_lst.map(extract_data).getInfo()

# Convert to Pandas DataFrame
data = []
for feature in features['features']:
    props = feature['properties']
    
    # Ensure the key exists and has a valid value
    if 'LST_Day_1km' in props and props['LST_Day_1km'] is not None:
        data.append([props['date'], props['LST_Day_1km']])

# Check if data exists before creating DataFrame
if data:
    df = pd.DataFrame(data, columns=['Date', 'LST_Day_1km'])

    # Convert LST from Kelvin to Celsius
    df['LST_Day_1km'] = df['LST_Day_1km'] * 0.02 - 273.15  # MODIS Scale Factor

    # Save dataset to a folder
    output_folder = "MODIS_LST_Dataset"
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, "MODIS_LST_Lankien.csv")

    df.to_csv(output_path, index=False)
    print(f"Data saved to {output_path}")
else:
    print("No valid LST data found for the specified location and time range.")


Data saved to MODIS_LST_Dataset/MODIS_LST_Lankien.csv


# NDVI data 16 day  

In [10]:
import ee
import pandas as pd
import os

# Authenticate and initialize Earth Engine
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project="gee-ndvi-berhe")

# Define location
latitude, longitude = 8.4281, 33.7833  # Lankien, South Sudan
roi = ee.Geometry.Point([longitude, latitude])

# Define time range
start_date = '2015-01-01'
end_date = '2024-12-31'

# Load MODIS NDVI data
ndvi_collection = ee.ImageCollection('MODIS/061/MOD13A1') \
    .select('NDVI') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi)

# Function to extract NDVI values
def extract_ndvi(image):
    date = image.date().format("YYYY-MM-dd")
    ndvi_value = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=500,
        bestEffort=True
    ).get('NDVI')

    return ee.Feature(None, {'date': date, 'NDVI': ndvi_value})

# Apply extraction
features = ndvi_collection.map(extract_ndvi).getInfo()

# Convert to Pandas DataFrame
data = []
for feature in features['features']:
    props = feature['properties']
    if 'NDVI' in props and props['NDVI'] is not None:
        data.append([props['date'], props['NDVI'] * 0.0001])  # MODIS Scale Factor

df = pd.DataFrame(data, columns=['Date', 'NDVI'])

# Save as CSV
output_folder = "NDVI_Dataset"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "MODIS_16day_NDVI_Lankien.csv")

df.to_csv(output_path, index=False)
print(f"NDVI data saved to {output_path}")


NDVI data saved to NDVI_Dataset/MODIS_NDVI_Lankien.csv


# 1. Resampling MODIS data to Weekly (with 16-day data)

In [12]:
import ee
import pandas as pd
import os

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project="gee-ndvi-berhe")

# Define location
latitude, longitude = 8.4281, 33.7833  # Lankien, South Sudan
roi = ee.Geometry.Point([longitude, latitude])

# Define time range
start_date = '2015-01-01'
end_date = '2024-12-31'

# Load MODIS NDVI data
ndvi_collection = ee.ImageCollection('MODIS/061/MOD13A1') \
    .select('NDVI') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi)

# Function to convert 16-day composite to weekly data (simple resampling)
def week_resample(image):
    # Get the date of the image
    date = image.date().format("YYYY-MM-dd")
    
    # Get the week of the year
    week_of_year = ee.Date(image.date()).get('week')
    
    # Return a feature with the date and week of the year
    return ee.Feature(None, {'date': date, 'week_of_year': week_of_year, 'NDVI': image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=500,
        bestEffort=True
    ).get('NDVI')})

# Apply extraction
features = ndvi_collection.map(week_resample).getInfo()

# Convert to Pandas DataFrame
data = []
for feature in features['features']:
    props = feature['properties']
    if 'NDVI' in props and props['NDVI'] is not None:
        data.append([props['date'], props['NDVI'] * 0.0001])  # MODIS Scale Factor

df = pd.DataFrame(data, columns=['Date', 'NDVI'])

# Save as CSV
output_folder = "Weekly_NDVI_Dataset"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "Weekly_MODIS_NDVI_Lankien.csv")

df.to_csv(output_path, index=False)
print(f"Weekly NDVI data saved to {output_path}")


Weekly NDVI data saved to Weekly_NDVI_Dataset/Weekly_MODIS_NDVI_Lankien.csv


# 2. Using Sentinel-2 NDVI Data for Weekly Temporal Resolution

In [14]:
import ee
import pandas as pd
import os

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project="gee-ndvi-berhe")

# Define location
latitude, longitude = 8.4281, 33.7833  # Lankien, South Sudan
roi = ee.Geometry.Point([longitude, latitude])

# Define time range
start_date = '2015-01-01'
end_date = '2024-12-31'

# Load Sentinel-2 NDVI data
sentinel_collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi) \
    .select(['B8', 'B4'])  # NIR and RED bands

# Function to calculate NDVI for Sentinel-2
def calculate_ndvi(image):
    # Compute NDVI using the NIR (B8) and Red (B4) bands
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    date = image.date().format("YYYY-MM-dd")
    
    # Reduce region to get the mean NDVI for the area of interest
    reduced = ndvi.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=30,
        maxPixels=1e8
    )
    
    # Return an ee.Feature with date and NDVI value
    return ee.Feature(None, {'date': date, 'NDVI': reduced.get('NDVI')})

# Apply NDVI calculation to the collection
ndvi_collection = sentinel_collection.map(calculate_ndvi)

# Convert the results to a list and fetch data
features = ndvi_collection.getInfo()

# Convert to Pandas DataFrame
data = []
for feature in features['features']:
    props = feature['properties']
    if 'NDVI' in props and props['NDVI'] is not None:
        data.append([props['date'], props['NDVI']])

df = pd.DataFrame(data, columns=['Date', 'NDVI'])

# Save as CSV
output_folder = "Weekly_NDVI_Sentinel_Dataset"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "Weekly_Sentinel_NDVI_Lankien.csv")

df.to_csv(output_path, index=False)
print(f"Weekly NDVI data saved to {output_path}")


Weekly NDVI data saved to Weekly_NDVI_Sentinel_Dataset/Weekly_Sentinel_NDVI_Lankien.csv
